# Using the PRIMO API

## Download a list of results from PRIMO

PRIMO API is <code>https://api-na.hosted.exlibrisgroup.com/primo/v1/pnxs?</code><br>

UI key = {YOUR KEY HERE}<br>

Scope options:
* All items = <code>everything</code>
* UI+Summit = <code>ui_alma_summit</code>
* UI Only = <code>ui_alma</code>

'vid' parameter is <code>UID</code>

Queries should begin with <code>q=</code>, then other terms, e.g. <code>q=any,contains,sage+grouse</code>

## Retrieve a specific record:

* <code>.../primo/v1/pnxs/{context}/{recordId}</code>
* context should be either L or PC
* recordId should be the recId

In [1]:
import requests
import pandas as pd
from time import sleep
import json

In [2]:
# Need a list of titles to search
# CSV of pnxId, titles
# Search by each title, save results to a file
# Find proquest records for each title, extract id, build URL

In [2]:
theses = pd.read_csv('./theses/2021-06-18_fishWildTheses.csv')

NameError: name 'pd' is not defined

In [1]:
theses

NameError: name 'theses' is not defined

In [14]:
col_list = ['setId', 'title']
theses = theses[col_list]

In [15]:
theses['results'] = ''

In [16]:
theses.head()

,setId,title,results
0,6bdbe0ecc13f00c85ec2660f02331c5b,Applications of geographical information syste...,
1,52455a4776dcd9b8827ed67547e655f1,The rainbow trout market in Mexico City,
2,eb81e31c11650f867c980b37d8340622,Reproductive ecology of tundra swans on the Ar...,
3,883f37f1097368babc93f7fcdc2541c2,"Seasonal movements, habitat use, and productiv...",
4,1cfe0576f821bec84cedac92565cfd1f,Winter ecology of migratory sage grouse and as...,


In [117]:
# Proquest Permanent URL Example:
# https://uidaho.idm.oclc.org/login?url=https://search.proquest.com/docview/2271755483

def getOutcome(results):
    outcome = []
    results = results.get('docs','Not Found')
    if results == 'Not Found':
        outcome.append('Not Found')
    else:
        for doc in results:
            link = doc['delivery']['GetIt1'][0]['links'][0]['link']
            if 'proquest' in link:
                #print(link)
                groups = link.split("/")
                #print(groups)
                groupsTwo = groups[4].split("?")
                pqNum = groupsTwo[0]
                link = 'https://uidaho.idm.oclc.org/login?url=https://search.proquest.com/docview/'+pqNum
                outcome.append(link)
            else:
                noPQ = "No Proquest Record Detected"
                outcome.append(noPQ)

    print(outcome)
    theses.at[index, 'results'] = outcome

In [ ]:
for index, row in theses.iterrows():
    if isinstance(row['results'], list):
        pass
    else:
        uidaho = {YOUR KEY HERE}
        scope = 'ui_alma' #'UI Only'
        title = str(row['title'])
        response = requests.get(f'https://api-na.hosted.exlibrisgroup.com/primo/v1/pnxs?vid=UID&scope={scope}&q=title,contains,"{title}",AND&apikey={uidaho}')
        print(response.status_code)
        print(index)
        getOutcome(response.json())
        sleep(0.5)

In [127]:
theses.iloc[18,:]

setId                       e5535355c309e64bb69f92dfbe8fe998
title      Ecology of moose on the Copper River Delta, Al...
results    [No Proquest Record Detected, https://uidaho.i...
Name: 18, dtype: object

In [128]:
for index, row in theses.iterrows():
    for res in row['results']:
        if 'docview' in res:
            theses.at[index, 'results'] = res
        else:
            pass

In [129]:
theses.to_csv('./theses/2021-06-28_fishWildAPI.csv', encoding='utf-8')